# Project Ryan: RL Trading Environment (Colab Starter)

Welcome to the Colab notebook for Project Ryan! This notebook will guide you through setting up, training, and evaluating the RL trading agent using the custom gym-mtsim-inspired environment.

In [ ]:
# SECTION 1: Install Required Packages
!pip install gymnasium stable-baselines3[extra] MetaTrader5 tqdm pandas numpy matplotlib plotly pathos scikit-learn

In [ ]:
# Detect if MetaTrader5 is available (for Colab/cloud compatibility)
import platform
MT5_AVAILABLE = False
if platform.system() == "Windows":
    try:
        import MetaTrader5 as mt5
        MT5_AVAILABLE = True
    except ImportError:
        print("MetaTrader5 not installed. Will use local cache or skip MT5 features.")
else:
    print("MetaTrader5 is only available on Windows. Running in cloud/offline mode.")

## Section 2: Clone the Project Repository

Clone the Project Ryan codebase from GitHub. If you have your own fork, update the URL below.

In [ ]:
# Clone the Project Ryan repo (replace with your fork if needed)
!git clone https://github.com/Traxin3/ryan-rl-trader.git
%cd ryan-rl-trader

## Section 3: Set Up Environment Variables and Configuration

Edit the configuration below to set your trading symbols, timeframes, and other environment parameters.

In [ ]:
# Set up your environment configuration here
SYMBOLS = ['EURUSD']
TIMEFRAMES = [1, 5, 15]
WINDOW_SIZE = 30
SYMBOL_MAX_ORDERS = 2
REWARD_SCALING = 1.0
MAX_LEVERAGE = 5.0
RISK_ADJUSTED_REWARD = True
EARLY_CLOSE_BONUS = 0.1
DIVERSIFICATION_BONUS = 0.05
# Add more config as needed

## Section 4: Download or Prepare Market Data

Download historical market data using the provided scripts. Data will be cached locally for fast reloads.

In [ ]:
# Download or prepare market data
# If your codebase has a script for this, run it here. Otherwise, data will be downloaded automatically on first run.
# Example (uncomment if you have a script):
# !python scripts/download_data.py --symbols EURUSD --timeframes 1 5 15

## Section 5: Run Training Script

Run the main training script to start training the RL agent in the custom environment.

In [ ]:
# Run the main training script
!python main.py

## Section 6: Monitor Training with TensorBoard

Launch TensorBoard below to monitor training progress and view logged metrics.

In [ ]:
# Launch TensorBoard in Colab
%load_ext tensorboard
%tensorboard --logdir ./tensorboard_logs/ --host 0.0.0.0

## Section 7: Evaluate and Visualize Results

After training, run the evaluation script or use the code below to analyze agent performance and visualize results.

In [ ]:
# Evaluate and visualize results after training
from stable_baselines3 import PPO
import numpy as np

# Reload the trained model
model = PPO.load("ppo_transformer_mtsim_final")

# You may need to re-create the environment as in main.py
from gym_mtsim.envs.mt_env import MtEnv
venv = MtEnv(
    symbols=SYMBOLS,
    timeframes=TIMEFRAMES,
    window_size=WINDOW_SIZE,
    symbol_max_orders=SYMBOL_MAX_ORDERS,
    reward_scaling=REWARD_SCALING,
    max_leverage=MAX_LEVERAGE,
    risk_adjusted_reward=RISK_ADJUSTED_REWARD,
    early_close_bonus=EARLY_CLOSE_BONUS,
    diversification_bonus=DIVERSIFICATION_BONUS
)
obs = venv.reset()
rewards = []
steps = 0

while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = venv.step(action)
    rewards.append(reward)
    steps += 1
    if done or truncated or steps >= 10000:
        break

print(f"Evaluation completed. Total reward: {sum(rewards):.2f} over {steps} steps")
print(f"Average reward per step: {np.mean(rewards):.4f}")

# Optionally, visualize performance (if supported)
try:
    venv.render(mode='advanced_figure')
except Exception as e:
    print(f"Rendering failed: {str(e)}")